#### 소프트맥스 함수 (softmax)
- 입력 값을 0~1 사이에 출력되도록 정규화하여 출력 값들의 총합이 항상 1이 되도록 함
- 보통 딥러닝에서 출력 노드의 활성화 함수로 많이 사용

In [8]:
import torch
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
    	super(Net, self).__init__()
    	self.hidden = torch.nn.Linear(n_feature, n_hidden) #은닉층
    	self.relu = torch.nn.ReLu(inplace=True)
    	self.out = torch.nn.Linear(n_hidden, n_output) #출력층
    	self.softmax = torch.nn.Softmax(dim=n_output)
    def forward(self, x):
    	x = self.hidden(x)
    	x = self.relu(x) #은닉층을 위한 렐루 활성화 함수
    	x = self.out(x)
    	x = self.softmax(x) #출력층을 위한 소프트맥스 활성화 함수
    	return x

#### 평균 제곱 오차 (MSE) 
- 실제 값과 예측 값의 차이(error)를 제곱하여 평균을 낸 것
- 작을 수록 → 예측력 good
- 회귀에서 손실 함수로 주로 사용

임시 모델 사용!

In [15]:
import torch.nn as nn
model = nn.Linear(in_features=1, out_features=1, bias=True)

In [26]:
x = torch.tensor([[1.0], [2.0], [3.0]])
y = torch.tensor([[2.0], [4.0], [6.0]])  

In [28]:
loss_fn = torch.nn.MSELoss(reduction='sum')
y_pred = model(x)
loss = loss_fn(y_pred, y)

#### 크로스 엔트로피 오차 (CEE)
- 분류 문제에서 원-핫 인코딩 했을 때만 사용할 수 있는 오차 계산법
- 일반적으로 분류 문제에서 데이터의 출력을 0과 1로 구분하기 위해 시그모이드 함수 사용 → 함수에 포함된 자연 상수 e 때문에 평균 제곱 오차 적용하면 울퉁불퉁한 그래프 출력 —> 크로스 엔트로피 손실 함수로 해결!
- CEE 적용 경우 경사 하강법 과정에서 학습이 지역 최소점에서 멈출 수 있음 → 방지하고자 자연 상수 e에 반대되는 자연 로그를 모델의 출력 값에 취함

In [37]:
loss = nn.CrossEntropyLoss()
input = torch.randn(5, 6, requires_grad=True) #torch.randn은 평균이 0이고 표준편차가 1인 가우시안 정규분포를 이용하여 숫자 생성
target = torch.empty(5, dtype=torch.long).random_(6) #torch.empty는 dtype torch.float32의 랜덤한 값으로 채워진 텐서를 반환
output = loss(input, target)
output.backward()

#### 드롭아웃(dropout)
- 학습 과정 중 임의로 일부 노드들을 학습에서 제외
- 과적합 해결

In [42]:
class DropoutModel(torch.nn.Module):
	def __init__(self):
		super(DropoutModel, self).__init__()
		self.layer1 = torch.nn.Linear(784, 1200)
		self.dropout1 = torch.nn.Dropout(0.5) #50%의 노드를 무작위로 선택해 사용하지 않겠다는 의미
		self.layer2 = torch.nn.Linear(1200, 1200)
		self.dropout2 = torch.nn.Dropout(0.5)
		self.layer3 = torch.nn.Linear(1200, 10)
		
	def forward(self, x):
		x = F.relu(self.layer1(x))
		x = self.dropout1(x)
		x = F.relu(self.layer2(x))
		x = self.dropout2(x)
		return self.layer3(x)

#### 미니 배치 경사 하강법 (mini-batch gradient descent) 
- 전체 데이터셋을 미니 배치 여러개로 나누고 → 미니 배치 한 개마다 기울기 구하고 → 그것의 평균 기울기 이용해 모델 업데이트해서 학습
- 전체 데이터 계산하는 것보다 빠름, 확률적 경사 하강법보다 안정적

In [51]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class CustomDataset(Dataset):
	def __init__(self):
		self.x_data = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
		self.y_data = [[12], [18], [11]]
	def __len__(self):
			return len(self.x_data)
	def __getitem__(self, idx):
			x = torch.FloatTensor(self.x_data[idx])
			y = torch.FloatTensor(self.y_data[idx])
			return x, y
dataset = CustomDataset()
dataloader = DataLoader(
	dataset, #데이터셋
	batch_size=2, #미니 배치 크기로 2의 제곱수를 사용
	shuffle=True, #데이터 불러올 때마다 랜덤으로 섞기
)

#### 아다그라드(Adagrad, Adaptive gradient)
- 변수(가중치)의 업데이트 횟수에 따라 학습률 조정하는 방법
- 많이 변화하지 않는 변수들의 학습률은 크게, 많이 변화하는 변수들의 학습률은 작게 함 → 많이 변화한 변수는 최적 값에 근접했을 것이라는 가정합에 작은 크기로 이동, 세밀하게 값 조정 → 적게 변화한 변수들 학습률 크게 하여 빠르게 오차 값 줄이고자 함

In [54]:
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.1) 

#### 아다델타(Adadelta, Adaptive delta)
- 아다그라드에서 G 값이 커짐에 따라 학습을 멈추는 문제 해결 위한 방법
- 아다그라드의 수식에서 학습률을 D 함수(가중치의 변화량 크기를 누적한 값)로 변환했기 때문에 학습률에 대한 하이퍼파라미터 필요 x

In [57]:
optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0) #학습률 기본값은 1.0

#### 알엠에스프롭(RMSProp)
- 아다그라드의 G(i) 값이 무한히 커지는 것을 방지하고자 제안된 방법 → G 함수에서 y(감마)만 추가

In [60]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01) #학습률 기본값 1e-2

#### 모멘텀(Momentum)
- 경사 하강법과 마찬가지로 매번 기울기 구하지만, 가중치를 수정하기 전에 이전 수정 방향(+,-)을 참고하여 같은 방향으로 일정한 비율 수정하는 방법
- 수정 방향 (+,-) —> 지그재그 현장 less + 일정 비율만큼 다음 값 결정하므로 관성 효과

In [63]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

#### 네스테로프 모멘텀(Nesterov Accelerated Gradient, NAG)
- 모멘텀 값이 적용된 지점에서 기울기 값을 계산
    - *모멘텀 값과 기울기 값이 더해져 실제 값을 만드는 기존 모멘텀과 달르다!*
- 모멘텀으로 절반 정도 이동 후 어떤 방식으로 이동해야 하는지 다시 계산하여 결정 → 모멘텀의 단점 극복 (멈추어야 할 시점에서도 관성에 의해 훨씬 멀리 갈 수 있다)
- 모멘텀 방법의 이점인 **빠른 이동 속도** 그대로 가져가면서 **멈추어야 할 적절한 시점**에서 **제공** 거는 데 훨씬 용이

In [66]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9,
						nesterov=True) #nesterov 기본값은 False

#### 아담(Adam, Adaptive Moment Estimation) 
